# Liquidity Token Analysis

## Code

### The code consists of three main parts:
- Defined queries: the queries that are called to the subgraph (each query is an entity)
- Update data functions: functions that call the subgraph and append the new data read to the current dataframes.
- Data manipulation/analysis functions: functions that manipulate the data to make desired calculations. 

### Instructions: Run the first 4 cells

In [ ]:
# Import the required libraries
import pandas as pd
import json
import requests
import pprint
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from matplotlib.ticker import StrMethodFormatter
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [ ]:

def defineUsersQuery(usersCount):
    usersQuery = '''
    {
      users (orderDirection: asc, first: 1000, skip: ''' + str(usersCount) + ''') {
        id
        usdSwapped
      }
    }
    '''
    return usersQuery

def defineShellTransfersQuery(transfersCount):
    shellTransfersQuery = '''
    {
      shellTransfers (orderDirection: asc, first: 1000, skip: ''' + str(transfersCount) + ''') {
        id
        timestamp
        block
        shellsTransfered
        from
        to
      }
    }
    '''
    return shellTransfersQuery

def defineDepositsQuery(depositsCount):
    depositsQuery = '''
    {
      deposits (orderDirection: asc, first: 1000, skip: ''' + str(depositsCount) + ''') {
        id
        timestamp
        block
        shellsMinted
        user {
          id
        }
      }
    }
    '''
    return depositsQuery

def defineWithdrawalsQuery(withdrawalsCount):
    withdrawalsQuery = '''
    {
      withdrawals (orderDirection: asc, first: 1000, skip: ''' + str(withdrawalsCount) + ''') {
        id
        timestamp
        block
        shellsBurnt
        user {
          id
        }
      }
    }
    '''
    return withdrawalsQuery

def getUser(x):
    return x['id']

def getCounts(x):
    return len(x)

def convertToDate(x):
    date = datetime.fromtimestamp(int(x)).date()
    day = date.day
    month = date.month
    if date.month < 10:
        month = '0'+str(date.month)
    if date.day < 10:
        day = '0'+str(date.day)
    date = f'{date.year}-{month}-{day}'
    return date

In [ ]:
def updateUserData(dataFrame, url):
    while True:
        count = len(dataFrame)
        query = defineUsersQuery(count)
        users = json.loads(requests.post(url, json={'query': query}).text)['data']['users']
        dataFrame = dataFrame.append(pd.DataFrame(users))
        if users == []:
            break
    return dataFrame.reset_index().drop(columns=['index'])


def updateShellTransfersData(dataFrame, url):
    while True:
        count = len(dataFrame)
        query = defineShellTransfersQuery(count)
        transfers = json.loads(requests.post(url, json={'query': query}).text)['data']['shellTransfers']
        if transfers == []:
            break
        newDF = pd.DataFrame(transfers)
        newDF['shellsTransfered'] = newDF['shellsTransfered'].astype(float)
        #newDF['date'] = newDF['timestamp'].apply(convertToDate)
        dataFrame = dataFrame.append(newDF)
    return dataFrame.reset_index().drop(columns=['index'])

    
def updateWithdrawalsData(dataFrame, url):
    while True:
        count = len(dataFrame)
        query = defineWithdrawalsQuery(count)
        withdrawals = json.loads(requests.post(url, json={'query': query}).text)['data']['withdrawals']
        if withdrawals == []:
            break
        newDF = pd.DataFrame(withdrawals)
        newDF['user'] = newDF['user'].apply(getUser)
        newDF['shellsBurnt'] = newDF['shellsBurnt'].astype(float)
        #newDF['date'] = newDF['timestamp'].apply(convertToDate)
        dataFrame = dataFrame.append(newDF)
    return dataFrame.reset_index().drop(columns=['index'])


def updateDepositsData(dataFrame, url):
    while True:
        count = len(dataFrame)
        query = defineDepositsQuery(count)
        deposits = json.loads(requests.post(url, json={'query': query}).text)['data']['deposits']
        if deposits == []:
            break
        newDF = pd.DataFrame(deposits)
        newDF['user'] = newDF['user'].apply(getUser)
        newDF['shellsMinted'] = newDF['shellsMinted'].astype(float)
        #newDF['date'] = newDF['timestamp'].apply(convertToDate)
        dataFrame = dataFrame.append(newDF)
    return dataFrame.reset_index().drop(columns=['index'])

In [ ]:
def createWithdrawalsVsDepositsChart(_withdrawalsDF, _depositsDF, poolStartDate, poolEndDate):
    w = _withdrawalsDF.drop(columns=['id', 'block', 'user'])
    w['date'] = w['timestamp'].apply(convertToDate)
    w = w[(w['date']>=poolStartDate)&(w['date']<=poolEndDate)]
    w = w.drop(columns=['timestamp'])
    w = w.groupby(['date']).sum()
    
    d = _depositsDF.drop(columns=['id', 'block', 'user'])
    d['date'] = d['timestamp'].apply(convertToDate)
    d = d[(d['date']>=poolStartDate)&(d['date']<=poolEndDate)]
    d = d.drop(columns=['timestamp'])
    d = d.groupby(['date']).sum()
    
    idx = pd.DataFrame()
    idx['date'] = pd.date_range(poolStartDate, poolEndDate)
    idx['date'] = idx['date'].astype(str).str[:10]
    
    merged = pd.merge(idx, w, on='date', how='outer')
    merged = pd.merge(merged, d, on='date', how='outer')
    merged['shellsMinted'] = merged['shellsMinted'].fillna(0)
    merged['shellsBurnt'] = merged['shellsBurnt'].fillna(0)
    merged = merged.set_index('date')
    merged['shellsBurnt'] = merged['shellsBurnt'].rolling(30).mean()
    merged['shellsMinted'] = merged['shellsMinted'].rolling(30).mean()
    
    sns.set(font_scale=1.4)
    merged.plot(figsize=(12, 10), linewidth=2.5)
    plt.gca().get_lines()[0].set_color("green")
    plt.gca().get_lines()[1].set_color("red")
    ax = plt.gca()
    leg = ax.get_legend()
    leg.legendHandles[0].set_color('red')
    leg.legendHandles[1].set_color('green')
    plt.xlabel("Date", labelpad=15)
    plt.xticks(rotation=30)
    plt.ylabel("Shells Movement", labelpad=15)
    ax.yaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))
    plt.title("Shells Movement Over Time (30-day Moving Average)", y=1.02, fontsize=22)
    
    
def createShellsSupplyChart(_withdrawalsDF, _depositsDF, poolStartDate, poolEndDate):
    w = _withdrawalsDF.drop(columns=['id', 'block', 'user'])
    w['date'] = w['timestamp'].apply(convertToDate)
    w = w[(w['date']>=poolStartDate)&(w['date']<=poolEndDate)]
    w = w.drop(columns=['timestamp'])
    w = w.groupby(['date']).sum()
    
    d = _depositsDF.drop(columns=['id', 'block', 'user'])
    d['date'] = d['timestamp'].apply(convertToDate)
    d = d[(d['date']>=poolStartDate)&(d['date']<=poolEndDate)]
    d = d.drop(columns=['timestamp'])
    d = d.groupby(['date']).sum()
    
    idx = pd.DataFrame()
    idx['date'] = pd.date_range(poolStartDate, poolEndDate)
    idx['date'] = idx['date'].astype(str).str[:10]
    
    merged = pd.merge(idx, w, on='date', how='outer')
    merged = pd.merge(merged, d, on='date', how='outer')
    merged['shellsMinted'] = merged['shellsMinted'].fillna(0)
    merged['shellsBurnt'] = merged['shellsBurnt'].fillna(0)
    merged['runningSumMints'] = merged['shellsMinted'].cumsum()
    merged['runningSumBurns'] = merged['shellsBurnt'].cumsum()
    merged['shellSupply'] = merged['runningSumMints'] - merged['runningSumBurns']
    merged = merged.drop(columns=['shellsBurnt', 'shellsMinted', 'runningSumBurns', 'runningSumMints'])
    merged = merged.set_index('date')
    
    sns.set(font_scale=1.4)
    merged.plot(figsize=(12, 10), linewidth=2.5, color='blue')
    plt.xlabel("Date", labelpad=15)
    plt.xticks(rotation=30)
    plt.ylabel("Shell Supply", labelpad=15)
    plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))
    plt.title("Shell Supply Over Time", y=1.02, fontsize=22)
    
    
def getTransferedShellsOverTime(_shellTransfersDF, poolStartDate, poolEndDate):
    t = _shellTransfersDF[(_shellTransfersDF['to']!='0x000000000000000000000000000000000000dead')]
    t['date'] = t['timestamp'].apply(convertToDate)
    t = t[(t['date']>=poolStartDate)&(t['date']<=poolEndDate)]
    t = t.drop(columns=['id', 'block', 'from', 'to', 'timestamp'])
    
    idx = pd.DataFrame()
    idx['date'] = pd.date_range(poolStartDate, poolEndDate)
    idx['date'] = idx['date'].astype(str).str[:10]
    
    merged = pd.merge(idx, t, left_on='date', right_on='date', how='outer')
    merged['shellsTransfered'] = merged['shellsTransfered'].fillna(0)
    merged['shellsTransfered'] = merged['shellsTransfered'].rolling(30).mean()
    merged = merged.set_index('date')
    
    sns.set(font_scale=1.4)
    merged.plot(figsize=(12, 10), linewidth=2.5, color='blue')
    plt.xlabel("Date", labelpad=15)
    plt.xticks(rotation=30)
    plt.ylabel("Shells Transfered", labelpad=15)
    plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))
    plt.title("Shells Transfered Between People Over Time (30-day Moving Average)", y=1.02, fontsize=22)


def plotTokenDays(_withdrawalsDF, _depositsDF, _shellTransfersDF, metric, poolStartDate, poolEndDate):
    w = _withdrawalsDF
    w['date'] = w['timestamp'].apply(convertToDate)
    w = w[(w['date']>=poolStartDate)&(w['date']<=poolEndDate)]
    w = w.drop(columns=['id', 'block', 'timestamp']).groupby(['date', 'user']).sum()
    w = w.reset_index()
    
    d = _depositsDF
    d['date'] = d['timestamp'].apply(convertToDate)
    d = d[(d['date']>=poolStartDate)&(d['date']<=poolEndDate)]
    d = d.drop(columns=['id', 'block', 'timestamp']).groupby(['date', 'user']).sum()
    d = d.reset_index()
    
    tFrom = _shellTransfersDF
    tFrom['date'] = tFrom['timestamp'].apply(convertToDate)
    tFrom = tFrom[(tFrom['date']>=poolStartDate)&(tFrom['date']<=poolEndDate)]
    tFrom = tFrom.drop(columns=['id', 'block', 'to', 'timestamp']).groupby(['date', 'from']).sum()
    tFrom = tFrom.rename(columns={'shellsTransfered': 'shellsSent'})
    
    tTo = _shellTransfersDF
    tTo = tTo.rename(columns={'shellsTransfered': 'shellsReceived'})
    tTo['date'] = tTo['timestamp'].apply(convertToDate)
    tTo = tTo[(tTo['date']>=poolStartDate)&(tTo['date']<=poolEndDate)]
    tTo = tTo.drop(columns=['id', 'block', 'from', 'timestamp']).groupby(['date', 'to']).sum()
    tTo = tTo.rename(columns={'shellsTransfered': 'shellsSent'})
    
    idx = pd.DataFrame()
    idx['date'] = pd.date_range(poolStartDate, poolEndDate)
    idx['date'] = idx['date'].astype(str).str[:10]
    
    merged = pd.merge(idx, w, left_on='date', right_on='date', how='outer')
    merged = pd.merge(merged, d, left_on=['date', 'user'], right_on=['date', 'user'], how='outer')
    merged = pd.merge(merged, tFrom, left_on=['date', 'user'], right_on=['date', 'from'], how='outer')
    merged = pd.merge(merged, tTo, left_on=['date', 'user'], right_on=['date', 'to'], how='outer')
    
    merged['shellsBurnt'] = merged['shellsBurnt'].fillna(0)
    merged['shellsMinted'] = merged['shellsMinted'].fillna(0)
    merged['shellsSent'] = merged['shellsSent'].fillna(0)
    merged['shellsReceived'] = merged['shellsReceived'].fillna(0)
    
    merged = merged.groupby(['date', 'user'], dropna=False).sum()
    merged = merged.reset_index()
   
    merged['dailyShells'] = merged['shellsMinted'] + merged['shellsReceived'] - merged['shellsBurnt'] - merged['shellsSent']
    merged = merged.drop(columns=['shellsMinted', 'shellsReceived', 'shellsBurnt', 'shellsSent'])
    
    pivot = merged.pivot(index='date', columns='user', values=['dailyShells']).fillna(0).cumsum()
    
    if metric == 'Mean':
        calculation = pivot.mean(axis=1)
    elif metric == 'Median':
        calculation = pivot.median(axis=1)
    elif metric == 'Percentiles':
        calculation = pivot.quantile([.1, .25, .5, .75], axis = 1)
        calculation = calculation.T
        
    sns.set(font_scale=1.4)
    calculation.plot(figsize=(12, 10), linewidth=2.5, color='blue')
    plt.xlabel("Date", labelpad=15)
    plt.xticks(rotation=30)
    plt.ylabel(f"{metric} Tokens", labelpad=15)
    plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:,.1f}'))
    plt.title(f"{metric} Token Days Over Time", y=1.02, fontsize=22)
    
    
def percentageOfTokenDays(_withdrawalsDF, _depositsDF, _shellTransfersDF, poolStartDate, poolEndDate):
    w = _withdrawalsDF
    w['date'] = w['timestamp'].apply(convertToDate)
    w = w[(w['date']>=poolStartDate)&(w['date']<=poolEndDate)]
    w = w.drop(columns=['id', 'block', 'timestamp']).groupby(['date', 'user']).sum()
    w = w.reset_index()
    
    d = _depositsDF
    d['date'] = d['timestamp'].apply(convertToDate)
    d = d[(d['date']>=poolStartDate)&(d['date']<=poolEndDate)]
    d = d.drop(columns=['id', 'block', 'timestamp']).groupby(['date', 'user']).sum()
    d = d.reset_index()
    
    tFrom = _shellTransfersDF[(_shellTransfersDF['to']!='0x000000000000000000000000000000000000dead')]
    tFrom['date'] = tFrom['timestamp'].apply(convertToDate)
    tFrom = tFrom[(tFrom['date']>=poolStartDate)&(tFrom['date']<=poolEndDate)]
    tFrom = tFrom.drop(columns=['id', 'block', 'to', 'timestamp']).groupby(['date', 'from']).sum()
    tFrom = tFrom.reset_index().rename(columns={'shellsTransfered': 'shellsSent', 'from': 'user'})

    tTo = _shellTransfersDF[(_shellTransfersDF['to']!='0x000000000000000000000000000000000000dead')]
    tTo = tTo.rename(columns={'shellsTransfered': 'shellsReceived'})
    tTo['date'] = tTo['timestamp'].apply(convertToDate)
    tTo = tTo[(tTo['date']>=poolStartDate)&(tTo['date']<=poolEndDate)]
    tTo = tTo.drop(columns=['id', 'block', 'from', 'timestamp']).groupby(['date', 'to']).sum()
    tTo = tTo.reset_index().rename(columns={'shellsTransfered': 'shellsSent', 'to':'user'})
    
    idx = pd.DataFrame()
    idx['date'] = pd.date_range(poolStartDate, poolEndDate)
    idx['date'] = idx['date'].astype(str).str[:10]
    
    merged = pd.merge(idx, w, left_on='date', right_on='date', how='outer')
    merged = pd.merge(merged, d, left_on=['date', 'user'], right_on=['date', 'user'], how='outer')
    #return merged
    merged = pd.merge(merged, tFrom, on=['date', 'user'], how='outer')
    #return merged.iloc[2790:]
    merged = pd.merge(merged, tTo, on=['date', 'user'], how='outer')
    #return merged
    
    merged['shellsBurnt'] = merged['shellsBurnt'].fillna(0)
    merged['shellsMinted'] = merged['shellsMinted'].fillna(0)
    merged['shellsSent'] = merged['shellsSent'].fillna(0)
    merged['shellsReceived'] = merged['shellsReceived'].fillna(0)
    
    merged = merged.groupby(['date', 'user'], dropna=False).sum()
    merged = merged.reset_index()
   
    merged['dailyShells'] = merged['shellsMinted'] + merged['shellsReceived'] - merged['shellsBurnt'] - merged['shellsSent']
    #return merged
    merged = merged.drop(columns=['shellsMinted', 'shellsReceived', 'shellsBurnt', 'shellsSent'])
    
    pivot = merged.pivot(index='date', columns='user', values=['dailyShells']).fillna(0).cumsum()
    percentage = pivot.div(pivot.sum(axis=1), axis=0)
    return percentage


def outstandingShells(_withdrawalsDF, _depositsDF, poolStartDate, poolEndDate):
    d = _depositsDF
    d['date'] = d['timestamp'].apply(convertToDate)
    d = d[(d['date']>=poolStartDate)&(d['date']<=poolEndDate)]
    d = d['shellsMinted'].sum()
    
    w = _withdrawalsDF
    w['date'] = w['timestamp'].apply(convertToDate)
    w = w[(w['date']>=poolStartDate)&(w['date']<=poolEndDate)]
    w = w['shellsBurnt'].sum()
    
    outstandingShells = d - w
    return outstandingShells


def calculateRewards(percentagesDF, rewardsByPeriod, period):
    dailyRewards = rewardsByPeriod/period
    rewards = percentagesDF*dailyRewards
    summedRewards = rewards.sum(axis=0)
    summedRewards = summedRewards[(summedRewards>0)]
    return summedRewards


def intersectionOfPools(_stablecoinRewards, _btcRewards):
    s20 = _stablecoinRewards.nlargest(20).reset_index().drop(columns=['level_0']).set_index('user').rename(columns={0:'Stablecoin Pool'})
    b20 = _btcRewards.nlargest(20).reset_index().drop(columns=['level_0']).set_index('user').rename(columns={0:'BTC Pool'})
    
    intersection = pd.merge(s20, b20, on='user', how='inner')
    intersection['total'] = intersection['Stablecoin Pool'] + intersection['BTC Pool']
    intersection['Percentage of Total Tokens'] = intersection['total']/(_stablecoinRewards.sum()+_btcRewards.sum())*100
    return intersection


def getBothRewards(_stablecoinWithdrawalsDF, _stablecoinDepositsDF, _stablecoinShellTransfersDF, 
                   _btcWithdrawalsDF, _btcDepositsDF, _btcShellTransfersDF, poolStartDate, poolEndDate):

    sP = percentageOfTokenDays(_stablecoinWithdrawalsDF, _stablecoinDepositsDF, _stablecoinShellTransfersDF, poolStartDate, poolEndDate)
    bP = percentageOfTokenDays(_btcWithdrawalsDF, _btcDepositsDF, _btcShellTransfersDF, poolStartDate, poolEndDate)
    
    percentageToStablecoin = int(input('What percentage of tokens would you want to allocate to the stablecoin pool?' ))
    tokens = int(input('What is the total number of tokens that you want to distribute in a given period? '))
    interfal = int(input('How long do you want the period to be in days? '))
    sTokens = tokens*(percentageToStablecoin/100)
    bTokens = tokens*(1-(percentageToStablecoin/100))
    sR = calculateRewards(sP, sTokens, interval).sort_values(ascending=False)
    bR = calculateRewards(bP, bTokens, interval).sort_values(ascending=False)
    stablecoinRewards = pd.DataFrame(sR).reset_index().drop(columns=['level_0']).rename(columns={0:'Stablecoin Pool'}).set_index('user')
    btcRewards = pd.DataFrame(bR).reset_index().drop(columns=['level_0']).rename(columns={0:'BTC Pool'}).set_index('user')
    merged = pd.merge(stablecoinRewards, btcRewards, on='user', how='outer').fillna(0)
    merged['total'] = merged['Stablecoin Pool'] + merged['BTC Pool']
    
    return merged


def getAllRewards(_stablecoinWithdrawalsDF, _stablecoinDepositsDF, _stablecoinShellTransfersDF, 
                   _btcWithdrawalsDF, _btcDepositsDF, _btcShellTransfersDF, 
                   _oStablecoinWithdrawalsDF, _oStablecoinDepositsDF, _oStablecoinShellTransfersDF,
                   _oBtcWithdrawalsDF, _oBtcDepositsDF, _oBtcShellTransfersDF,
                   oPoolStartDate, oPoolEndDate, cPoolStartDate, cPoolEndDate):

    cSP = percentageOfTokenDays(_stablecoinWithdrawalsDF, _stablecoinDepositsDF, _stablecoinShellTransfersDF, cPoolStartDate, cPoolEndDate)
    cBP = percentageOfTokenDays(_btcWithdrawalsDF, _btcDepositsDF, _btcShellTransfersDF, cPoolStartDate, cPoolEndDate)
    oSP = percentageOfTokenDays(_oStablecoinWithdrawalsDF, _oStablecoinDepositsDF, _oStablecoinShellTransfersDF, oPoolStartDate, oPoolEndDate)
    oBP = percentageOfTokenDays(_oBtcWithdrawalsDF, _oBtcDepositsDF, _oBtcShellTransfersDF, oPoolStartDate, oPoolEndDate)
    
    percentageToStablecoin = int(input('What percentage of tokens would you want to allocate to the stablecoin pool?' ))
    tokens = int(input('What is the total number of tokens that you want to distribute in a given period? '))
    interfal = int(input('How long do you want the period to be in days? '))
    sTokens = tokens*(percentageToStablecoin/100)
    bTokens = tokens*(1-(percentageToStablecoin/100))
    
    cSR = calculateRewards(cSP, sTokens, interval).sort_values(ascending=False)
    cBR = calculateRewards(cBP, bTokens, interval).sort_values(ascending=False)
    oSR = calculateRewards(oSP, sTokens, interval).sort_values(ascending=False)
    oBR = calculateRewards(oBP, bTokens, interval).sort_values(ascending=False)
    
    currentStablecoinRewards = pd.DataFrame(cSR).reset_index().drop(columns=['level_0']).rename(columns={0:'Stablecoin Pool'}).set_index('user')
    currentBtcRewards = pd.DataFrame(cBR).reset_index().drop(columns=['level_0']).rename(columns={0:'BTC Pool'}).set_index('user')
    originalStablecoinRewards = pd.DataFrame(oSR).reset_index().drop(columns=['level_0']).rename(columns={0:'Original Stablecoin Pool'}).set_index('user')
    originalBtcRewards = pd.DataFrame(oBR).reset_index().drop(columns=['level_0']).rename(columns={0:'Original BTC Pool'}).set_index('user')
    
    merged = pd.merge(currentStablecoinRewards, currentBtcRewards, on='user', how='outer').fillna(0)
    merged = pd.merge(merged, originalStablecoinRewards, on='user', how='outer').fillna(0)
    merged = pd.merge(merged, originalBtcRewards, on='user', how='outer').fillna(0)
    merged['total'] = merged['Stablecoin Pool'] + merged['BTC Pool'] + merged['Original Stablecoin Pool'] + merged['Original BTC Pool']
    
    return merged


def intersectionOfAllPools(_stablecoinRewards, _btcRewards, _oStableCoinRewards, _oBtcRewards):
    cS20 = _stablecoinRewards.nlargest(20).reset_index().drop(columns=['level_0']).set_index('user').rename(columns={0:'Stablecoin Pool'})
    cB20 = _btcRewards.nlargest(20).reset_index().drop(columns=['level_0']).set_index('user').rename(columns={0:'BTC Pool'})
    oS20 = _oStableCoinRewards.nlargest(20).reset_index().drop(columns=['level_0']).set_index('user').rename(columns={0:'Original Stablecoin Pool'})
    oB20 = _oBtcRewards.nlargest(20).reset_index().drop(columns=['level_0']).set_index('user').rename(columns={0:'Original BTC Pool'})
    
    intersection = pd.merge(cS20, cB20, on='user', how='inner')
    intersection = pd.merge(intersection, oS20, on='user', how='inner')
    intersection = pd.merge(intersection, oB20, on='user', how='inner')
    intersection['total'] = intersection['Stablecoin Pool'] + intersection['BTC Pool'] + intersection['Original Stablecoin Pool'] + intersection['Original BTC Pool']
    intersection['Percentage of Total Tokens'] = intersection['total']/(_stablecoinRewards.sum()+_btcRewards.sum()+_oStableCoinRewards.sum()+_oBtcRewards.sum())*100
    return intersection


def generatePieChart(rewardsSeries):
    top10 = pd.DataFrame(rewardsSeries[:10])
    other = pd.DataFrame(pd.Series(rewardsSeries[10:].sum())).rename(columns={0:'total'}, index={0: 'Other'}).reset_index().set_index('index')
    concat = pd.concat([top10, other], axis=0)
    plt.pie(concat['total'], labels=concat.index, labeldistance=None, radius=3, autopct='%1.1f%%')
    plt.legend(labels=concat.index, prop={'size': 11}, bbox_to_anchor=(0.0, 0.1))
    plt.title('Share of Tokens Distributed of Top 10 Users vs The Rest', pad=180)
    plt.show()

## Get Data

### Code
- This part gets the data from the graphs and stores it at dataframes. 
- Each entity of the liquidity pool has its own dataframe. 

### Instructions: run the cells below to get the most recent data into the application.

In [ ]:
# URLs for each pool
stablecoinPool = 'https://api.thegraph.com/subgraphs/name/mhanono27/shell-dai-usdc-usdt-susd'
btcPool = 'https://api.thegraph.com/subgraphs/name/mhanono27/shell-wbtc-renbtc-sbtc'
oStablecoinPool = 'https://api.thegraph.com/subgraphs/name/mhanono27/shell-dai-usdc-usdt-susd-original'
oBtcPool = 'https://api.thegraph.com/subgraphs/name/mhanono27/shell-wbtc-renbtc-sbtc-original'

In [ ]:
# Define datetimes for pools
currentPoolsStartDate = '2020-12-28'
currentPoolsEndDate = datetime.today().strftime('%Y-%m-%d')
originalPoolsStartDate = '2020-10-04'
originalPoolsEndDate = '2020-12-28'

In [ ]:
# Initialize dataframes for entities (only run once).
stablecoinUsersDF = pd.DataFrame()
stablecoinShellTransfersDF = pd.DataFrame()
stablecoinWithdrawalsDF = pd.DataFrame()
stablecoinDepositsDF = pd.DataFrame()

btcUsersDF = pd.DataFrame()
btcShellTransfersDF = pd.DataFrame()
btcWithdrawalsDF = pd.DataFrame()
btcDepositsDF = pd.DataFrame()

oStablecoinUsersDF = pd.DataFrame()
oStablecoinShellTransfersDF = pd.DataFrame()
oStablecoinWithdrawalsDF = pd.DataFrame()
oStablecoinDepositsDF = pd.DataFrame()

oBtcUsersDF = pd.DataFrame()
oBtcShellTransfersDF = pd.DataFrame()
oBtcWithdrawalsDF = pd.DataFrame()
oBtcDepositsDF = pd.DataFrame()

In [ ]:
# Update data from stablecoin pool
stablecoinUsersDF = updateUserData(stablecoinUsersDF, stablecoinPool)
stablecoinShellTransfersDF = updateShellTransfersData(stablecoinShellTransfersDF, stablecoinPool)
stablecoinWithdrawalsDF = updateWithdrawalsData(stablecoinWithdrawalsDF, stablecoinPool)
stablecoinDepositsDF = updateDepositsData(stablecoinDepositsDF, stablecoinPool)

In [ ]:
# Update data from BTC pool
btcUsersDF = updateUserData(btcUsersDF ,btcPool)
btcShellTransfersDF = updateShellTransfersData(btcShellTransfersDF ,btcPool)
btcWithdrawalsDF = updateWithdrawalsData(btcWithdrawalsDF ,btcPool)
btcDepositsDF = updateDepositsData(btcDepositsDF ,btcPool)

In [ ]:
# Update data from original stablecoin pool
oStablecoinUsersDF = updateUserData(oStablecoinUsersDF, oStablecoinPool)
oStablecoinShellTransfersDF = updateShellTransfersData(oStablecoinShellTransfersDF, oStablecoinPool)
oStablecoinWithdrawalsDF = updateWithdrawalsData(oStablecoinWithdrawalsDF, oStablecoinPool)
oStablecoinDepositsDF = updateDepositsData(oStablecoinDepositsDF, oStablecoinPool)

In [ ]:
# Update data from original BTC pool
oBtcUsersDF = updateUserData(oBtcUsersDF ,oBtcPool)
oBtcShellTransfersDF = updateShellTransfersData(oBtcShellTransfersDF ,oBtcPool)
oBtcWithdrawalsDF = updateWithdrawalsData(oBtcWithdrawalsDF ,oBtcPool)
oBtcDepositsDF = updateDepositsData(oBtcDepositsDF ,oBtcPool)

## Stablecoin Pool

- Run the cells in order to get the desired results

### Supply of Shells Over Time

In [ ]:
createShellsSupplyChart(stablecoinWithdrawalsDF, stablecoinDepositsDF, currentPoolsStartDate, currentPoolsEndDate)

### Withdrawals vs Deposits Over Time

In [ ]:
createWithdrawalsVsDepositsChart(stablecoinWithdrawalsDF, stablecoinDepositsDF, currentPoolsStartDate, currentPoolsEndDate)

### Shells Trasfered Between Wallets Over Time

In [ ]:
getTransferedShellsOverTime(stablecoinShellTransfersDF, currentPoolsStartDate, currentPoolsEndDate)

### MeanToken Days Over Time

In [ ]:
plotTokenDays(stablecoinWithdrawalsDF, stablecoinDepositsDF, stablecoinShellTransfersDF, 'Mean', currentPoolsStartDate, currentPoolsEndDate)

### Median Token Days Over Time

In [ ]:
plotTokenDays(stablecoinWithdrawalsDF, stablecoinDepositsDF, stablecoinShellTransfersDF, 'Median', currentPoolsStartDate, currentPoolsEndDate)

### Current Amount Of Outstanding Shells

In [ ]:
outstandingShells(stablecoinWithdrawalsDF, stablecoinDepositsDF, currentPoolsStartDate, currentPoolsEndDate)

### Percentage of Token Days by User

- tokens: Set the amount of tokens that will be given in an interval
- interval: the length of time for the interval (days)

In [ ]:
p = percentageOfTokenDays(stablecoinWithdrawalsDF, stablecoinDepositsDF, stablecoinShellTransfersDF, currentPoolsStartDate, currentPoolsEndDate)

In [ ]:
tokens = int(input('How many tokens would you like to distribute per period? '))
interval = int(input('How many days would you like the periods to last? '))
print('Continue...')
stablecoinRewards = calculateRewards(p, tokens, interval).sort_values(ascending=False)

In [ ]:
total = int(stablecoinRewards.sum())
print('Total tokens distributed:',total)

### Top N Users and Their Amount of Tokens

In [ ]:
n = int(input('How many top users would you like to see? '))
stablecoinRewards[:n]

### Top % of Users Share of Total Tokens Distributed

In [ ]:
percentage = int(input('What percentage of users would you like to calculate for? '))
users = int(percentage*len(stablecoinRewards)/100)
percentageOfTokens = int(stablecoinRewards[:users].sum()/stablecoinRewards.sum()*100)
print(f'{percentage}% of users ({users} users) would hold {percentageOfTokens}% of tokens distributed.')

### Statistics on Distributed Tokens

In [ ]:
print('Mean: ',stablecoinRewards.mean())
print('Stdv: ',stablecoinRewards.std())
print('Min: ',stablecoinRewards.min())
print('Max: ',stablecoinRewards.max())
print('\nPercentiles: ')
print('25%:', stablecoinRewards.quantile(.25))
print('50%:', stablecoinRewards.quantile(.5))
print('75%:', stablecoinRewards.quantile(.75))

### Histogram

In [ ]:
stablecoinRewards.hist(bins=np.arange(0,200,10))

### Pie Chart

In [ ]:
rClean = pd.DataFrame(stablecoinRewards).reset_index().drop(columns=['level_0']).rename(columns={0:'total'}).set_index('user')['total']
generatePieChart(rClean)

## BTC Pool
- Run the cells in order to get the desired results

### Supply of Shells Over Time

In [ ]:
createShellsSupplyChart(btcWithdrawalsDF, btcDepositsDF, currentPoolsStartDate, currentPoolsEndDate)

### Withdrawals vs Deposits Over Time

In [ ]:
createWithdrawalsVsDepositsChart(btcWithdrawalsDF, btcDepositsDF, currentPoolsStartDate, currentPoolsEndDate)

### Shells Trasfered Between Wallets Over Time

In [ ]:
getTransferedShellsOverTime(btcShellTransfersDF, currentPoolsStartDate, currentPoolsEndDate)

### MeanToken Days Over Time

In [ ]:
plotTokenDays(btcWithdrawalsDF, btcDepositsDF, btcShellTransfersDF, 'Mean', currentPoolsStartDate, currentPoolsEndDate)

### Median Token Days Over Time

In [ ]:
plotTokenDays(btcWithdrawalsDF, btcDepositsDF, btcShellTransfersDF, 'Median', currentPoolsStartDate, currentPoolsEndDate)

### Current Amount Of Outstanding Shells

In [ ]:
outstandingShells(btcWithdrawalsDF, btcDepositsDF, currentPoolsStartDate, currentPoolsEndDate)

### Percentage of Token Days by User

- tokens: Set the amount of tokens that will be given in an interval
- interval: the length of time for the interval (days)

In [ ]:
p = percentageOfTokenDays(btcWithdrawalsDF, btcDepositsDF, btcShellTransfersDF, currentPoolsStartDate, currentPoolsEndDate)

In [ ]:
tokens = int(input('How many tokens would you like to distribute per period? '))
interval = int(input('How many days would you like the periods to last? '))
print('Continue...')
btcRewards = calculateRewards(p, tokens, interval).sort_values(ascending=False)

In [ ]:
total = int(btcRewards.sum())
print('Total tokens distributed:',total)

### Top N Users and Their Amount of Tokens

In [ ]:
n = int(input('How many top users would you like to see? ')) 
btcRewards[:n]

### Top % of Users Share of Total Tokens Distributed

In [ ]:
percentage = int(input('What percentage of users would you like to calculate for? '))
users = int(percentage*len(btcRewards)/100)
percentageOfTokens = int(btcRewards[:users].sum()/btcRewards.sum()*100)
print(f'{percentage}% of users ({users} users) would hold {percentageOfTokens}% of tokens distributed.')

### Statistics on Distributed Tokens

In [ ]:
print('Mean: ',btcRewards.mean())
print('Stdv: ',btcRewards.std())
print('Min: ',btcRewards.min())
print('Max: ',btcRewards.max())
print('\nPercentiles: ')
print('25%:', btcRewards.quantile(.25))
print('50%:', btcRewards.quantile(.5))
print('75%:', btcRewards.quantile(.75))

### Histogram

In [ ]:
btcRewards.hist(bins=np.arange(0,500,10))

### Pie Chart

In [ ]:
rClean = pd.DataFrame(btcRewards).reset_index().drop(columns=['level_0']).rename(columns={0:'total'}).set_index('user')['total']
generatePieChart(rClean)

## Overlap Between Current Stablecoin and BTC Pools
- The analysis here is done by combining the retroactive rewards for both pools. 
- You will be prompted for the different parameters that determine the token distribution. 

In [ ]:
combinedPoolRewards = getBothRewards(stablecoinWithdrawalsDF, stablecoinDepositsDF, stablecoinShellTransfersDF, 
                   btcWithdrawalsDF, btcDepositsDF, btcShellTransfersDF, currentPoolsStartDate, currentPoolsEndDate)
currentCombinedRewards = combinedPoolRewards['total'].sort_values(ascending=False)

### Number of Users Who Appear on Both

In [ ]:
bothCount = combinedPoolRewards[(combinedPoolRewards['Stablecoin Pool']>0)&(combinedPoolRewards['BTC Pool']>0)]
count = len(bothCount)
percentageOfTotal = int(len(bothCount)/len(currentCombinedRewards)*100)
print(f'{count} users appear in both pools ({percentageOfTotal}% of total).')

### Intersection of Users in Top 20 Users for Both Pools 

In [ ]:
intersectionOfPools(stablecoinRewards, btcRewards)

### Statistics on Distributed Tokens

In [ ]:
print('Mean: ',currentCombinedRewards.mean())
print('Stdv: ',currentCombinedRewards.std())
print('Min: ',currentCombinedRewards.min())
print('Max: ',currentCombinedRewards.max())
print('\nPercentiles: ')
print('25%:', currentCombinedRewards.quantile(.25))
print('50%:', currentCombinedRewards.quantile(.5))
print('75%:', currentCombinedRewards.quantile(.75))

### Histogram

In [ ]:
currentCombinedRewards.hist(bins=np.arange(0,100,10))

### Pie Chart

In [ ]:
generatePieChart(currentCombinedRewards)

## Original Stablecoin Pool

- Run the cells in order to get the desired results

### Supply of Shells Over Time

In [ ]:
createShellsSupplyChart(oStablecoinWithdrawalsDF, oStablecoinDepositsDF, originalPoolsStartDate, originalPoolsEndDate)

### Withdrawals vs Deposits Over Time

In [ ]:
createWithdrawalsVsDepositsChart(oStablecoinWithdrawalsDF, oStablecoinDepositsDF, originalPoolsStartDate, originalPoolsEndDate)

### Shells Trasfered Between Wallets Over Time

In [ ]:
getTransferedShellsOverTime(oStablecoinShellTransfersDF, originalPoolsStartDate, originalPoolsEndDate)

### MeanToken Days Over Time

In [ ]:
plotTokenDays(oStablecoinWithdrawalsDF, oStablecoinDepositsDF, oStablecoinShellTransfersDF, 'Mean', originalPoolsStartDate, originalPoolsEndDate)

### Median Token Days Over Time

In [ ]:
plotTokenDays(oStablecoinWithdrawalsDF, oStablecoinDepositsDF, oStablecoinShellTransfersDF, 'Median', originalPoolsStartDate, originalPoolsEndDate)

### Current Amount Of Outstanding Shells

In [ ]:
outstandingShells(oStablecoinWithdrawalsDF, oStablecoinDepositsDF, originalPoolsStartDate, originalPoolsEndDate)

### Percentage of Token Days by User

- tokens: Set the amount of tokens that will be given in an interval
- interval: the length of time for the interval (days)

In [ ]:
p = percentageOfTokenDays(oStablecoinWithdrawalsDF, oStablecoinDepositsDF, oStablecoinShellTransfersDF, originalPoolsStartDate, originalPoolsEndDate)

In [ ]:
tokens = int(input('How many tokens would you like to distribute per period? '))
interval = int(input('How many days would you like the periods to last? '))
print('Continue...')
originalStablecoinRewards = calculateRewards(p, tokens, interval).sort_values(ascending=False)

In [ ]:
total = int(originalStablecoinRewards.sum())
print('Total tokens distributed:',total)

### Top N Users and Their Amount of Tokens

In [ ]:
n = int(input('How many top users would you like to see? '))
originalStablecoinRewards[:n]

### Top % of Users Share of Total Tokens Distributed

In [ ]:
percentage = int(input('What percentage of users would you like to calculate for? '))
users = int(percentage*len(originalStablecoinRewards)/100)
percentageOfTokens = int(originalStablecoinRewards[:users].sum()/originalStablecoinRewards.sum()*100)
print(f'{percentage}% of users ({users} users) would hold {percentageOfTokens}% of tokens distributed.')

### Statistics on Distributed Tokens

In [ ]:
print('Mean: ',originalStablecoinRewards.mean())
print('Stdv: ',originalStablecoinRewards.std())
print('Min: ',originalStablecoinRewards.min())
print('Max: ',originalStablecoinRewards.max())
print('\nPercentiles: ')
print('25%:', originalStablecoinRewards.quantile(.25))
print('50%:', originalStablecoinRewards.quantile(.5))
print('75%:', originalStablecoinRewards.quantile(.75))

### Histogram

In [ ]:
originalStablecoinRewards.hist(bins=np.arange(0,200,10))

### Pie Chart

In [ ]:
rClean = pd.DataFrame(originalStablecoinRewards).reset_index().drop(columns=['level_0']).rename(columns={0:'total'}).set_index('user')['total']
generatePieChart(rClean)

## Original BTC Pool
- Run the cells in order to get the desired results

### Supply of Shells Over Time

In [ ]:
createShellsSupplyChart(oBtcWithdrawalsDF, oBtcDepositsDF, originalPoolsStartDate, originalPoolsEndDate)

### Withdrawals vs Deposits Over Time

In [ ]:
createWithdrawalsVsDepositsChart(oBtcWithdrawalsDF, oBtcDepositsDF, originalPoolsStartDate, originalPoolsEndDate)

### Shells Trasfered Between Wallets Over Time

In [ ]:
getTransferedShellsOverTime(oBtcShellTransfersDF, originalPoolsStartDate, originalPoolsEndDate)

### MeanToken Days Over Time

In [ ]:
plotTokenDays(oBtcWithdrawalsDF, oBtcDepositsDF, oBtcShellTransfersDF, 'Mean', originalPoolsStartDate, originalPoolsEndDate)

### Median Token Days Over Time

In [ ]:
plotTokenDays(oBtcWithdrawalsDF, oBtcDepositsDF, oBtcShellTransfersDF, 'Median', originalPoolsStartDate, originalPoolsEndDate)

### Current Amount Of Outstanding Shells

In [ ]:
outstandingShells(oBtcWithdrawalsDF, oBtcDepositsDF, originalPoolsStartDate, originalPoolsEndDate)

### Percentage of Token Days by User

- tokens: Set the amount of tokens that will be given in an interval
- interval: the length of time for the interval (days)

In [ ]:
p = percentageOfTokenDays(oBtcWithdrawalsDF, oBtcDepositsDF, oBtcShellTransfersDF, originalPoolsStartDate, originalPoolsEndDate)

In [ ]:
tokens = int(input('How many tokens would you like to distribute per period? '))
interval = int(input('How many days would you like the periods to last? '))
print('Continue...')
originalBtcRewards = calculateRewards(p, tokens, interval).sort_values(ascending=False)

In [ ]:
total = int(originalBtcRewards.sum())
print('Total tokens distributed:',total)

### Top N Users and Their Amount of Tokens

In [ ]:
n = int(input('How many top users would you like to see? '))
originalBtcRewards[:n]

### Top % of Users Share of Total Tokens Distributed

In [ ]:
percentage = int(input('What percentage of users would you like to calculate for? '))
users = int(percentage*len(originalBtcRewards)/100)
percentageOfTokens = int(originalBtcRewards[:users].sum()/originalBtcRewards.sum()*100)
print(f'{percentage}% of users ({users} users) would hold {percentageOfTokens}% of tokens distributed.')

### Statistics on Distributed Tokens

In [ ]:
print('Mean: ',originalBtcRewards.mean())
print('Stdv: ',originalBtcRewards.std())
print('Min: ',originalBtcRewards.min())
print('Max: ',originalBtcRewards.max())
print('\nPercentiles: ')
print('25%:', originalBtcRewards.quantile(.25))
print('50%:', originalBtcRewards.quantile(.5))
print('75%:', originalBtcRewards.quantile(.75))

### Histogram

In [ ]:
originalBtcRewards.hist(bins=np.arange(0,500,10))

### Pie Chart

In [ ]:
rClean = pd.DataFrame(originalBtcRewards).reset_index().drop(columns=['level_0']).rename(columns={0:'total'}).set_index('user')['total']
generatePieChart(rClean)

## Overlap Between Original Stablecoin and BTC Pools
- The analysis here is done by combining the retroactive rewards for both pools. 
- You will be prompted for the different parameters that determine the token distribution. 

In [ ]:
combinedPoolRewards = getBothRewards(oStablecoinWithdrawalsDF, oStablecoinDepositsDF, oStablecoinShellTransfersDF, 
                   oBtcWithdrawalsDF, oBtcDepositsDF, oBtcShellTransfersDF, originalPoolsStartDate, originalPoolsEndDate)
originalCombinedRewards = combinedPoolRewards['total'].sort_values(ascending=False)

### Number of Users Who Appear on Both

In [ ]:
bothCount = combinedPoolRewards[(combinedPoolRewards['Stablecoin Pool']>0)&(combinedPoolRewards['BTC Pool']>0)]
count = len(bothCount)
percentageOfTotal = int(len(bothCount)/len(originalCombinedRewards)*100)
print(f'{count} users appear in both pools ({percentageOfTotal}% of total).')

### Intersection of Users in Top 20 Users for Both Pools 

In [ ]:
intersectionOfPools(originalStablecoinRewards, originalBtcRewards)

### Statistics on Distributed Tokens

In [ ]:
print('Mean: ',originalCombinedRewards.mean())
print('Stdv: ',originalCombinedRewards.std())
print('Min: ',originalCombinedRewards.min())
print('Max: ',originalCombinedRewards.max())
print('\nPercentiles: ')
print('25%:', originalCombinedRewards.quantile(.25))
print('50%:', originalCombinedRewards.quantile(.5))
print('75%:', originalCombinedRewards.quantile(.75))

### Histogram

In [ ]:
originalCombinedRewards.hist(bins=np.arange(0,100,10))

### Pie Chart

In [ ]:
generatePieChart(originalCombinedRewards)

## Overlap Between Complete Stablecoin and BTC Pools
- The analysis here is done by combining the retroactive rewards for both pools. 
- You will be prompted for the different parameters that determine the token distribution. 

In [ ]:
combinedPoolRewards = getAllRewards(stablecoinWithdrawalsDF, stablecoinDepositsDF, stablecoinShellTransfersDF, 
                                   btcWithdrawalsDF, btcDepositsDF, btcShellTransfersDF, 
                                   oStablecoinWithdrawalsDF, oStablecoinDepositsDF, oStablecoinShellTransfersDF,
                                   oBtcWithdrawalsDF, oBtcDepositsDF, oBtcShellTransfersDF,
                                   originalPoolsStartDate, originalPoolsEndDate, currentPoolsStartDate, currentPoolsEndDate)
allCombinedRewards = combinedPoolRewards['total'].sort_values(ascending=False)

###### Number of Users Who Appear on Both

In [ ]:
bothCount = combinedPoolRewards[(combinedPoolRewards['Stablecoin Pool']>0)&(combinedPoolRewards['BTC Pool']>0)]
count = len(bothCount)
percentageOfTotal = int(len(bothCount)/len(allCombinedRewards)*100)
print(f'{count} users appear in all pools ({percentageOfTotal}% of total).')

### Intersection of Users in Top 20 Users for Both Pools 

In [ ]:
intersectionOfAllPools(stablecoinRewards, btcRewards, originalStablecoinRewards, originalBtcRewards)

### Statistics on Distributed Tokens

In [ ]:
print('Mean: ',allCombinedRewards.mean())
print('Stdv: ',allCombinedRewards.std())
print('Min: ',allCombinedRewards.min())
print('Max: ',allCombinedRewards.max())
print('\nPercentiles: ')
print('25%:', allCombinedRewards.quantile(.25))
print('50%:', allCombinedRewards.quantile(.5))
print('75%:', allCombinedRewards.quantile(.75))

### Histogram

In [ ]:
allCombinedRewards.hist(bins=np.arange(0,100,10))

### Pie Chart

In [ ]:
generatePieChart(allCombinedRewards)